In [0]:
!mkdir ~/.kaggle && echo '{"username":"polinarevina","key":"5bb914194d8b5103c7fab5f72b7c1806"}' >~/.kaggle/kaggle.json 

In [2]:
!ls

drive  sample_data


In [3]:
!kaggle competitions download -c recsys-iad-challenge

 99% 232M/235M [00:02<00:00, 67.4MB/s]
100% 235M/235M [00:02<00:00, 90.7MB/s]
 97% 356M/368M [00:04<00:00, 48.0MB/s]
100% 368M/368M [00:04<00:00, 77.6MB/s]
  0% 0.00/10.9M [00:00<?, ?B/s]
100% 10.9M/10.9M [00:00<00:00, 101MB/s]


In [4]:
!unzip items.json.zip
!unzip random_benchmark.csv.zip
!unzip train.json.zip

Archive:  items.json.zip
  inflating: items.json              
Archive:  random_benchmark.csv.zip
  inflating: random_benchmark.csv    
Archive:  train.json.zip
  inflating: train.json              


In [0]:
import json 
from scipy.sparse import coo_matrix
import pandas as pd
import numpy as np
from tqdm.auto import tqdm 

In [8]:
%%time
items_list=[]
with tqdm(open('./items.json')) as inf:
    for line in inf:
        item=json.loads(line)
        if isinstance(item['image'], float):
            item['image']=[0 for _ in range(96)]
            
        item['image']=np.array(item['image'])
        items_list.append(item)

items=pd.DataFrame(items_list).set_index('itemId')


CPU times: user 12.6 s, sys: 1.12 s, total: 13.7 s
Wall time: 13.7 s


In [0]:
len(items)

328050

In [20]:
items.head()

,content,image,title
itemId,,,
0,"Согласитесь, дорогие любители собак, до чего ж...","[[-0.169], [0.129], [0.067], [0.019], [0.281],...",Пять забавных «морщинистых» пород собак
1,"Контуры Третьей Поперечной улицы, состоявшей ...","[[-0.158], [-0.112], [-0.325], [0.05], [-0.114...",История улицы Ирининской в Гомеле
2,Источник: http://infodays.ru Вообще он как-то ...,"[[0.084], [-0.181], [0.008], [0.34], [-0.03], ...",Зачем Дудь всё время спрашивает гостей програм...
3,41-летняя Светлана Зейналова решила окрестить ...,"[0.034, -0.119, -0.062, 0.025, 0.128, -0.041, ...",Светлана Зейналова крестила младшую дочь
4,«Организованные преступные группировки ГБАО де...,"[-0.061, -0.015, -0.198, -0.047, 0.054, 0.029,...","ГКНБ: бандиты в ГБАО делают вид, что рассталис..."


In [0]:
items.as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([['Согласитесь, дорогие любители собак, до чего же мило и обаятельно выглядят собаки с забавными складками кожи – морщинками! Некоторые такие породы в свое время были остромодными и, соответственно, дефицитными. Цена – не радовала… И все-таки, как полагают наши британские и испанские коллеги, складки-морщинки – знак собачьей нежности и красоты, не больше и не меньше. Давайте с ними согласимся и заодно посмотрим на подборку из пяти «морщинистых» пород.\n  Столь забавные складки-морщинки, как вы понимаете, появляются у собак не с возрастом и не от старости, как можно было бы подумать. В первую очередь, они возникают из-за генетической предрасположенности, обусловленной большим содержанием в организме так называемой гиалуроновой кислоты. Именно она «ответственна» за столь интересную, мясистую и в глубоких складках кожу. Кстати, мимические морщины у собак тоже бывают, прямо как и у людей, но сегодня речь не о них – а о тех очаровательных складках, которые сводят с ума поклонников дан

In [0]:
# lower 
items["content"] = items["content"].apply(lambda x: x.lower())
items["title"] = items["title"].apply(lambda x: x.lower())

In [0]:
import string
def delete_punctuation(x):
    punctuation = list(string.punctuation)
    return ''.join([a if a not in punctuation + ['\n'] else ' ' for a in x])

In [0]:
items["content"] = items["content"].apply(delete_punctuation)
items["title"] = items["title"].apply(delete_punctuation)

In [0]:
items.to_csv("prepr_items.csv")

In [0]:
!ls

items.json	 random_benchmark.csv	   spacy-ru    train.json.zip
items.json.zip	 random_benchmark.csv.zip  test.csv
prepr_items.csv  sample_data		   train.json


In [0]:
# new_items = pd.read_csv("prepr_items.csv", lineterminator='\n')

In [0]:
# new_items.head()

In [0]:
pip install pymorphy2==0.8

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 7.8MB/s 


In [0]:
pip install -U pymorphy2-dicts-ru

     |████████████████████████████████| 8.0MB 2.9MB/s 


In [0]:
# text lemmatization 
import pymorphy2 

morph = pymorphy2.MorphAnalyzer() 

In [0]:
morph.normal_forms("словечки")[0]

'словечко'

In [0]:
alls = []
for row in tqdm(new_items["title"]):
  # print(row)
  news = []
  for word in row.split():
    new = morph.normal_forms(word)[0]
    news.append(new)
  alls.append(" ".join(news))
# print(alls)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
alls[:10]

['пять забавный «морщинистых» порода собака',
 'история улица ирининский в гомель',
 'зачем дудь весь время спрашивать гость программа о секс',
 'светлана зейналов крестить младший дочь',
 'гкнб бандит в гбао делать вид что расстаться с прошлое',
 'победа люба цена шесть «грязных» приём из бокс',
 'мечта',
 'этот сладость признать полезный для здоровье',
 'миронов похоже что отечественный чиновник существовать в параллельный реальность',
 'левша роман глава 4']

In [0]:
a = pd.DataFrame(alls)

In [0]:
a.head()

,0
0,пять забавный «морщинистых» порода собака
1,история улица ирининский в гомель
2,зачем дудь весь время спрашивать гость програм...
3,светлана зейналов крестить младший дочь
4,гкнб бандит в гбао делать вид что расстаться с...


In [0]:
a.to_csv("alls_titles.csv")

In [0]:
alls2 = []
for row in tqdm(new_items["content"]):
  # print(row)
  news = []
  for word in str(row).split():
    new = morph.normal_forms(word)[0] 
    news.append(new)
  alls2.append(" ".join(news))
# print(alls)

In [0]:
feat_it = pd.DataFrame({"titles_pr": alls, "contents_pr": alls2})

In [0]:
feat_it.head()

,titles_pr,contents_pr
0,пять забавный «морщинистых» порода собака,согласиться дорогой любитель собака до что же ...
1,история улица ирининский в гомель,контур три поперечный улица состоять до недавн...
2,зачем дудь весь время спрашивать гость програм...,источник http infodays ru вообще он как то сам...
3,светлана зейналов крестить младший дочь,41 летний светлана зейналов решить окрестить 5...
4,гкнб бандит в гбао делать вид что расстаться с...,«организовать преступный группировка гбао дела...


In [0]:
# def clean_text(text):
#     ## Remove stop words
#     stops = set(stopwords.words("english"))
#     text = [w for w in text if not w in stops and len(w) >= 3]
    
#     return text

In [0]:
!ls

alls_contents.csv  alls_titles.csv  drive  sample_data


In [0]:
import pandas as pd

In [0]:
titles = pd.read_csv("/content/drive/My Drive/rec syst/alls_titles.csv")

In [0]:
titles["0"]

0                 пять забавный «морщинистых» порода собака
1                         история улица ирининский в гомель
2         зачем дудь весь время спрашивать гость програм...
3                   светлана зейналов крестить младший дочь
4         гкнб бандит в гбао делать вид что расстаться с...
                                ...                        
328045    в новороссийск начаться ремонтный восстановите...
328046    рецепт крабовый рулет отличный закуска для нов...
328047    видеокарта nvidia rtx 2080 ti часто выходить и...
328048                              необъяснимый и страшный
328049    несколько лёгкое рецепт ликёр из черноплодный ...
Name: 0, Length: 328050, dtype: object

In [0]:
contents = pd.read_csv("/content/drive/My Drive/rec syst/alls_contents.csv")

In [0]:
contents["0"].values

array(['согласиться дорогой любитель собака до что же мило и обаятельно выглядеть собака с забавный складка кожа – морщинка некоторый такой порода в свой время быть остромодный и соответственно дефицитный цена – не радовала… и весь таки как полагать наш британский и испанский коллега складка морщинка – знак собачий нежность и красота не большой и не маленький давать с они согласиться и заодно посмотреть на подборка из пять «морщинистых» порода столь забавный складка морщинка как вы понимать появляться у собака не с возраст и не от старость как можно быть бы подумать в один очередь они возникать из за генетический предрасположенность обусловить больший содержание в организм так называть гиалуронов кислота именно она «ответственна» за столь интересный мясистый и в глубокий складка кожа кстати мимический морщина у собака тоже бывать прямо как и у человек но сегодня речь не о они – а о тот очаровательный складка который сводить с ум поклонник дать порода сразу оговориться ухаживать за кожн

In [0]:
import pandas as pd

contents = pd.read_csv("/content/drive/My Drive/rec syst/alls_contents.csv")
titles = pd.read_csv("/content/drive/My Drive/rec syst/alls_titles.csv")

In [0]:
import re
new_titles = []

for text in titles["0"]:
  new = re.sub(r"[a-zA-Z]+|[\d]+|[^\w]+", " ", str(text))
  new_titles.append(new)

contents_titles = []

for text in contents["0"]:
  new = re.sub(r"[a-zA-Z]+|[\d]+|[^\w]+", " ", str(text))
  contents_titles.append(new)

In [0]:
import nltk 
nltk.download("stopwords")
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("russian")

In [0]:
stop_titles = []
for sent in tqdm(new_titles):
  a = []
  for i in str(sent).split():
    if i not in cachedStopWords: 
      a.append(i)
  stop_titles.append(" ".join(a))

stop_contents = []
for sent in tqdm(contents_titles):
  a = []
  for i in str(sent).split():
    if i not in cachedStopWords: 
      a.append(i)
  stop_contents.append(" ".join(a))

In [0]:
allss = pd.DataFrame({"titles_s":stop_titles, "contents_s":stop_contents})

In [0]:
allss.to_csv("no_stops.csv")

In [0]:
no_stops = pd.read_csv("/content/drive/My Drive/rec syst/no_stops.csv")

In [5]:
no_stops.shape

(328050, 3)

In [6]:
no_stops.head()

,Unnamed: 0,titles_s,contents_s
0,0,пять забавный «морщинистых» порода собака,согласиться дорогой любитель собака мило обаят...
1,1,история улица ирининский гомель,контур поперечный улица состоять недавний врем...
2,2,дудь весь время спрашивать гость программа секс,источник http infodays ru вообще отвечать вопр...
3,3,светлана зейналов крестить младший дочь,41 летний светлана зейналов решить окрестить 5...
4,4,гкнб бандит гбао делать вид расстаться прошлое,«организовать преступный группировка гбао дела...


In [0]:
image_vect = np.vstack(items.image)

In [0]:
import scipy.sparse

In [0]:
image_vect_src = scipy.sparse.csr_matrix(image_vect)

I used TFIDF vectorizer to extract features from text collection. It converts a collection of text documents to a matrix of token counts. This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix, which are needed for LightFM model.

In [0]:
# Countvectorizer does not normalizes results 
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vectorizer = CountVectorizer()

In [0]:
titles_vectorized = vectorizer.fit_transform(no_stops["titles_s"].values.astype('str'))

In [0]:
# have a look at features 
print(vectorizer.get_feature_names())
# print(titles_vectorized.toarray())

['00', '000', '000р', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '1000', '10000', '101', '102', '105', '11', '110', '112', '12', '120', '1200', '128', '13', '130', '138', '14', '140', '142', '148', '15', '150', '1500', '16', '160', '16th', '17', '170', '18', '180', '19', '1917', '1939', '1941', '1942', '1943', '1944', '1945', '1960', '1961', '1967', '1970', '1980', '1988', '1990', '1991', '1992', '1993', '1996', '1998', '1999', '1ой', '20', '200', '2000', '2004', '2007', '2008', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2024', '2025', '207', '2070', '2077', '2080', '21', '22', '220', '229', '23', '230', '24', '25', '250', '26', '27', '28', '29', '29м', '30', '300', '3000', '31', '32', '33', '34', '340', '35', '350', '3500', '36', '360', '37', '38', '39', '3d', '40', '400', '4000', '41', '42', '43', '44', '45', '450', '46', '47', '48', '49', '4x4', '4х4', '50', '500', '5000', '51', '52', '53', '54', '

In [0]:
# # we could also extracted N-grams as features 
# vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
# titles_ngrams2 = vectorizer2.fit_transform(items.title)
# # print(vectorizer2.get_feature_names())

In [0]:
# TfidfVectorizer normalizes results 
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vect_title = TfidfVectorizer()
titles_vectorized = vect_title.fit_transform(no_stops["titles_s"].values.astype('str'))

In [0]:
vect_content = TfidfVectorizer()
content_vectorized = vect_content.fit_transform(no_stops["contents_s"].values.astype("str"))

In [0]:
sample=pd.read_csv('random_benchmark.csv')

In [0]:
sample.head(2)

,userId,itemId
0,1,242249
1,1,117266


In [16]:
# As lightfm has a coo-sparse matrix as an input, we should convert our data 

# https://stackoverflow.com/questions/57346498/transforming-json-into-coo-sparse-matrix-in-python 

users = {}
items = {}

data = [] # non-zero values in matrix  
items_ind = [] #row indices  
users_ind = [] #column indices  
num = 0   
us = 0
with tqdm(open("./train.json")) as file:   
  for line in file:  
    j_line = json.loads(line)  

    for item, rating in j_line["trainRatings"].items(): #going through extracted values
      items_ind.append(int(item))
      data.append((-1) ** (int(rating) + 1))    
      users_ind.append(int(num))  
    num+=1

In [0]:
print(num)

2489


In [0]:
# creating COO sparse matrix from 3 arrays 
train_data=coo_matrix((data,(users_ind, items_ind)))

In [19]:
train_data.shape 

(42977, 328050)

In [0]:
# matrix info. seems legit!  

print("User-item interactions matrix shape is: {}".format(train_data.shape))
print("Non-zero values: {}".format(train_data.nnz))
print("Share of non-zero values: {}".format(train_data.nnz / train_data.shape[0] / train_data.shape[1]))

User-item interactions matrix shape is: (42976, 328050)
Non-zero values: 6714489
Share of non-zero values: 0.00047626309904767934


In [0]:
print(train_data)

  (0, 180165)	0
  (0, 286761)	1
  (0, 127012)	1
  (0, 117072)	0
  (0, 86362)	0
  (0, 4023)	0
  (0, 165772)	0
  (0, 323309)	0
  (0, 206875)	0
  (0, 195382)	0
  (0, 51285)	0
  (0, 130646)	0
  (0, 241457)	0
  (0, 324148)	0
  (0, 75541)	0
  (0, 181162)	0
  (0, 266283)	0
  (0, 61514)	0
  (0, 30447)	0
  (0, 223325)	0
  (0, 262038)	0
  (0, 84861)	0
  (0, 205115)	0
  (0, 173150)	0
  (0, 46138)	0
  :	:
  (4348, 33587)	0
  (4348, 237668)	0
  (4348, 320886)	0
  (4348, 246442)	0
  (4348, 283707)	0
  (4348, 12808)	0
  (4348, 229806)	0
  (4348, 322019)	0
  (4348, 272165)	0
  (4348, 127275)	0
  (4348, 255029)	0
  (4348, 315502)	0
  (4348, 95956)	0
  (4348, 27757)	0
  (4348, 300760)	0
  (4348, 141034)	0
  (4348, 23594)	0
  (4348, 237742)	0
  (4348, 187039)	0
  (4348, 282766)	0
  (4348, 212700)	0
  (4348, 113138)	0
  (4348, 198779)	0
  (4348, 125091)	0
  (4348, 290905)	0


In [0]:
from scipy.sparse import hstack, identity
item_features = hstack([titles_vectorized, content_vectorized], format="csr")

In [25]:
item_features.shape 

(328050, 666546)

In [8]:
type(content_vectorized)

scipy.sparse.csr.csr_matrix

In [21]:
pip install lightfm

In [0]:
import lightfm 
model=lightfm.LightFM(loss='logistic',random_state=42)

In [0]:
import time   

In [0]:
%%time 
start = time.time() 
model.fit_partial(train_data, item_features=item_features, epochs=10)
  
print("We built LightFM in {} seconds".format(time.time() - start))

In [0]:
sample

,userId,itemId
0,1,242249
1,1,117266
2,1,123441
3,1,148575
4,1,155695
...,...,...
3018181,42975,4202
3018182,42975,111136
3018183,42975,195676
3018184,42975,292604


In [0]:
sample = pd.read_csv("random_benchmark.csv")
sample["pred"] = model.predict(
    sample.userId.values,
    sample.itemId.values,
    item_features=item_features)
sample.sort_values(["userId", "pred"], ascending=[True, False], inplace=True)
sample.drop(columns=["pred"], inplace=True)
sample.to_csv("lightfm_tfidf2.csv", index=False)

In [38]:
! kaggle competitions submit -c recsys-iad-challenge -m 'Test' -f lightfm_tfidf2.csv

100% 35.7M/35.7M [00:03<00:00, 10.5MB/s]
Successfully submitted to Articles RecSys